<a href="https://colab.research.google.com/github/AaronRaj06/SummerInternIIIT-H/blob/main/YOLO_Road_Damage_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Analyze the provided Roboflow dataset for road damage detection (RDD2022, Indian data subset) and train a YOLOv8 model to detect road damage. Provide the code directly in a Colab notebook, taking inspiration from "https://colab.research.google.com/drive/17szx5Cryeshhi9fKQin7Qw3whJ-piaKC?usp=sharing". The dataset details are: Roboflow workspace "yolo-omwe2", project "rdd2022-esodt-3xvig", version 2, format "yolov12". The Roboflow API key will be provided separately.

## Install and import libraries

### Subtask:
Install necessary libraries like `roboflow`, `ultralytics`, and others. Import required modules.


**Reasoning**:
The first step is to install the necessary libraries, `roboflow` and `ultralytics`, using pip.



In [2]:
%pip install roboflow ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 M

**Reasoning**:
The next step is to import the necessary modules, `roboflow` and `YOLO` from `ultralytics`.



## Download Dataset

### Subtask:
Use the provided Roboflow API key and project details to download the dataset in YOLOv8 format.

In [4]:
from roboflow import Roboflow
from google.colab import userdata

# Retrieve the API key from Colab secrets
api_key = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=api_key)

project = rf.workspace("yolo-omwe2").project("rdd2022-esodt-3xvig")
version = project.version(2)
dataset = version.download("yolov12")
dataset_dir = dataset.location

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to RDD2022-2 in yolov12:: 100%|██████████| 76771/76771 [01:17<00:00, 990.87it/s]  


## Load and Explore Data

### Subtask:
Load the downloaded dataset and explore its structure and contents to understand the annotations and image data.

In [5]:
import os

# List the contents of the dataset directory
print(f"Dataset downloaded to: {dataset_dir}")
print("Contents of the dataset directory:")
for item in os.listdir(dataset_dir):
    print(item)

# You can further explore subdirectories like train, valid, test, etc.
# Example: List contents of the train directory
train_dir = os.path.join(dataset_dir, 'train')
if os.path.exists(train_dir):
    print(f"\nContents of the train directory: {train_dir}")
    for item in os.listdir(train_dir):
        print(item)

Dataset downloaded to: /content/RDD2022-2
Contents of the dataset directory:
train
data.yaml
README.roboflow.txt
README.dataset.txt
test

Contents of the train directory: /content/RDD2022-2/train
labels
images


## Model Training

### Subtask:
Choose a suitable object detection model (like YOLOv8, but using Yolov12 as discussed) and train it on the downloaded dataset.

In [1]:
from ultralytics import YOLO
import os
import yaml

# Load a YOLOv8n model
# If you want to resume from a specific checkpoint, you can provide the path to the weights file
# Otherwise, 'yolov8n.pt' will load the pre-trained model
model = YOLO('yolov8n.pt')

# Define the path to the data.yaml file
data_yaml_path = os.path.join(dataset_dir, 'data.yaml')

# Load the data.yaml file
with open(data_yaml_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

# Update relative paths to absolute paths
data_yaml['train'] = os.path.join(dataset_dir, data_yaml['train'])
# Temporarily set val path to train path to satisfy Ultralytics requirement
data_yaml['val'] = os.path.join(dataset_dir, data_yaml['train']) # WORKAROUND: Using train data for validation

if 'test' in data_yaml:
    data_yaml['test'] = os.path.join(dataset_dir, data_yaml['test'])

# Write the updated data.yaml back to the file
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

print(f"Updated data.yaml with absolute paths and workaround for validation: {data_yaml_path}")

# Print the contents of the modified data.yaml to verify
with open(data_yaml_path, 'r') as f:
    modified_data_yaml = yaml.safe_load(f)
print("\nContents of modified data.yaml:")
print(yaml.dump(modified_data_yaml, indent=2))

# Train the model
# Pass the path to the modified data.yaml file
# Set epochs to 50 and resume=True to continue training from the last checkpoint
results = model.train(data=data_yaml_path, epochs=50, resume=True) # Changed epochs to 50 and added resume=True

ModuleNotFoundError: No module named 'ultralytics'

In [8]:
import yaml
import os

data_yaml_path = os.path.join(dataset_dir, 'data.yaml')

if os.path.exists(data_yaml_path):
    with open(data_yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)
    print("Contents of data.yaml:")
    print(yaml.dump(data_yaml, indent=2))
else:
    print(f"Error: {data_yaml_path} not found.")

Contents of data.yaml:
names:
- '0'
- '1'
- '2'
- '3'
- '4'
nc: 5
roboflow:
  license: CC BY 4.0
  project: rdd2022-esodt-3xvig
  url: https://universe.roboflow.com/yolo-omwe2/rdd2022-esodt-3xvig/dataset/2
  version: 2
  workspace: yolo-omwe2
test: ../test/images
train: ../train/images
val: ../valid/images



In [13]:
import os

# Define source and destination directories
source_dir = dataset_dir
dest_dir = '/content/'

# Create symbolic links for train, valid, and test image directories
for subdir in ['train', 'valid', 'test']:
    source_path = os.path.join(source_dir, subdir, 'images')
    dest_path = os.path.join(dest_dir, subdir, 'images') # Link to /content/train/images etc.

    # Ensure the destination directory exists
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)

    # Create symbolic link
    if os.path.exists(source_path):
        if not os.path.exists(dest_path):
            os.symlink(source_path, dest_path)
            print(f"Created symbolic link from {source_path} to {dest_path}")
        else:
            print(f"Symbolic link already exists at {dest_path}")
    else:
        print(f"Source directory not found: {source_path}")

Created symbolic link from /content/RDD2022-2/train/images to /content/train/images
Source directory not found: /content/RDD2022-2/valid/images
Created symbolic link from /content/RDD2022-2/test/images to /content/test/images
